In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np

#######################################################################################################
#######################################################################################################

# S8 : Traits

## Spatial Network

In [ ]:
df_proximity = pd.read_csv('../Data_sets/Proximity_county_network.csv',index_col = None)

In [ ]:
spatial_network = nx.from_pandas_edgelist(df_proximity, source = 'county_origin', target = 'county_destination')
spatial_network.remove_edges_from(list(nx.selfloop_edges(spatial_network)))    

## Hesitancy

In [ ]:
df10 = pd.read_csv('../Data_sets/US_County_hesitancy.csv', index_col=False)

In [ ]:
df_hesitancy = df10[df10['Year'] == 2015][['FIPS','Hesitancy Level']]

## Network

In [ ]:
# REMOVE COUNTIES IN HESITANCY NOT IN SPATIAL NETWORK
spatial_network.remove_nodes_from([each_node for each_node in list(df_hesitancy['FIPS']) if each_node not in  spatial_network.nodes()])

In [ ]:
# REMOVE COUNTIES IN SPATIAL NETWORK NOT IN HESITANCY NOT 
spatial_network.remove_nodes_from([each_node for each_node in spatial_network.nodes() if each_node not in list(df_hesitancy['FIPS'])])

In [ ]:
nodes = list(df_hesitancy['FIPS'])
hesitancy_dict = dict(zip(nodes, list(df_hesitancy['Hesitancy Level'])))
status_dict = dict(zip(nodes, ['Vulnerable' if each_hesitancy >= 0.05 else 'Protected' for each_node, each_hesitancy in hesitancy_dict.items()]))

In [ ]:
# ASIGN ATTRIBUTES TO COUNTIES
nx.set_node_attributes(spatial_network, hesitancy_dict, "Hesitancy")
nx.set_node_attributes(spatial_network, status_dict, "Status")

## Measure neighboring influence 

In [ ]:
multiplier = np.percentile(np.array(df_hesitancy['Hesitancy Level']),95)

In [ ]:
multiplier = np.percentile(np.array(df_hesitancy['Hesitancy Level']),95)
fraction = [ sum([(spatial_network.nodes[each_neighbor]['Hesitancy']) for each_neighbor in spatial_network.neighbors(each_node)])  / spatial_network.degree(each_node) / multiplier for each_node in spatial_network.nodes()]
nodes = [ spatial_network.nodes[each_node]['Status'] for each_node in spatial_network.nodes()]

In [ ]:
df_neighboring = pd.DataFrame(
    {'County Status': nodes,
     'Neighboring level of influence': fraction,
    })


In [ ]:
colors = ['#9171D1','#71B7D1']
sns.set_palette(sns.color_palette(colors))
ax = sns.pointplot(x = "County Status", y = "Neighboring level of influence", hue = 'County Status', linestyle = '', data = df_neighboring, dodge=True)
plt.grid(axis='y')
plt.legend(title=False,loc="upper left",fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("County status",fontsize=14)
plt.ylabel("Neighboring level of influence",fontsize=14)
ax.figure.savefig('SF-influence-neighboring.png',transparent=True,dpi=200)